In [ ]:
def pertubate(self):
    epsilon_min = self.config['perturbation']['epsilon_min']
    epsilon_max = self.config['perturbation']['epsilon_max']
    epsilon_steps = self.config['perturbation']['steps']

    epsilon_values = np.linspace(epsilon_min, epsilon_max, epsilon_steps) # aralık oluştur

    accus, mafs = list(), list()  # her epsilon için

    # döngü
    for epsilon in tqdm.tqdm(epsilon_values):
        prediction_ = []
        real_ = []

        # testte döngü
        for data, label in self.test:
            data = data.cuda() if tr.cuda.is_available() else data
            label = label.cuda() if tr.cuda.is_available() else label
            real_.append(label)

            if epsilon != 0:
                data.requires_grad = True
                prediction = self.net(data)
                loss = self.loss_function(prediction, label)
                loss.backward()
                perturbation = epsilon * data.grad.data.sign()
                x_adv = tr.clamp(data + perturbation, -1, 1)
            else:
                x_adv = data

            prediction = self.net(x_adv)
            prediction_.append(prediction.detach().cpu())

        prediction_ = tr.cat(prediction_, 0)
        real_ = tr.cat(real_, 0)
        prediction_ = tr.argmax(prediction_, -1)
        accu = self.accu_(prediction_, real_)
        maf1 = self.maf1(predicted=prediction_, real=real_)
        accus.append(accu)
        mafs.append(maf1)

    print("Doğruluk Skorları:", accus)
    print("Macro F1 Skorları:", mafs)

In [ ]:
epsilon_values = np.linspace(
    self.config['perturbation']['epsilon_min'],
    self.config['perturbation']['epsilon_max'],
    self.config['perturbation']['steps']
)

for epsilon in tqdm.tqdm(epsilon_values):
    prediction_ = []
    real_ = []

In [ ]:
def to_device(tensor):
    return tensor.cuda() if tr.cuda.is_available() else tensor

for data, label in self.test:
    data = to_device(data)
    label = to_device(label)
    real_.append(label)

In [ ]:
def apply_perturbation(data, label, epsilon, model, loss_function):
    if epsilon != 0:
        data.requires_grad = True
        prediction = model(data)
        loss = loss_function(prediction, label)
        loss.backward()

        perturbation = epsilon * data.grad.data.sign()

        x_adv = tr.clamp(data + perturbation, -1, 1)
    else:
        x_adv = data

    return x_adv

In [ ]:
prediction = make_prediction(self.net, x_adv)
prediction_.append(prediction)

In [ ]:
prediction_ = tr.cat(prediction_, 0)
real_ = tr.cat(real_, 0)

accu, maf1 = calculate_metrics(prediction_, real_, self.accu_, self.maf1)

accus.append(accu)
mafs.append(maf1)

In [ ]:
import numpy as np
import tqdm
import torch as tr

def generate_epsilon_values(config):
    return np.linspace(
        config['perturbation']['epsilon_min'],
        config['perturbation']['epsilon_max'],
        config['perturbation']['steps']
    )

def process_data(data, label):
    data = data.cuda() if tr.cuda.is_available() else data
    label = label.cuda() if tr.cuda.is_available() else label
    return data, label

def apply_perturbation(data, label, epsilon, model, loss_function):
    if epsilon != 0:
        data.requires_grad = True
        prediction = model(data)
        loss = loss_function(prediction, label)
        loss.backward()
        perturbation = epsilon * data.grad.data.sign()
        x_adv = tr.clamp(data + perturbation, -1, 1)
    else:
        x_adv = data
    return x_adv

def calculate_metrics(predictions, labels, accu_fn, maf1_fn):
    predictions = tr.argmax(predictions, -1)
    accu = accu_fn(predictions, labels)
    maf1 = maf1_fn(predicted=predictions, real=labels)
    return accu, maf1

def pertubate(self):
    accus, mafs = list(), list()
    epsilon_values = generate_epsilon_values(self.config)

    for epsilon in tqdm.tqdm(epsilon_values):
        prediction_ = []
        real_ = []

        for data, label in self.test:
            data, label = process_data(data, label)
            real_.append(label)

            x_adv = apply_perturbation(data, label, epsilon, self.net, self.loss_function)
            prediction = self.net(x_adv)
            prediction_.append(prediction.detach().cpu())

        prediction_ = tr.cat(prediction_, 0)
        real_ = tr.cat(real_, 0)

        accu, maf1 = calculate_metrics(prediction_, real_, self.accu_, self.maf1)
        accus.append(accu)
        mafs.append(maf1)

    print("Accuracy scores :", accus)
    print("macro f1 scores:", mafs)